# Capitulo 02 Instalacao Configuracao

Notebook gerado automaticamente a partir do código fonte python.


In [ ]:
# -*- coding: utf-8 -*-
"""
capitulo_02_instalacao_configuracao
"""

# capitulo_02_instalacao_configuracao
import duckdb
import os

In [ ]:
import duckdb

con = duckdb.connect(config={"allow_unsigned_extensions": "true"})

### Permitindo Extensões Não Assinadas

In [ ]:
import duckdb

con = duckdb.connect()

# Instalar extensão da comunidade
con.install_extension("h3", repository="community")

# Carregar extensão
con.load_extension("h3")

## 10. Extensões do DuckDB

Instalando e carregando extensões.

In [ ]:
import duckdb

r1 = duckdb.sql("SELECT 42 AS i")
duckdb.sql("SELECT i * 2 AS k FROM r1").show()

## 9. Relações Temporárias

Usando resultados de queries como tabelas temporárias.

In [ ]:
import duckdb

# ✅ CORRETO: Cada thread tem sua própria conexão
def boa_pratica():
    con = duckdb.connect()
    con.sql("SELECT 1").fetchall()

# ❌ INCORRETO: Compartilhando conexão entre threads
conexao_global = duckdb.connect()

def ma_pratica():
    # Evite usar conexão global em múltiplas threads
    return conexao_global.sql("SELECT 1").fetchall()

## 8. Boas Práticas com Threads

Exemplos de uso correto e incorreto de conexões em múltiplas threads.

In [ ]:
import duckdb

# Múltiplas configurações
con = duckdb.connect(config={
    'threads': 4,
    'max_memory': '4GB',
    'default_order': 'DESC'
})

### Múltiplas Configurações

In [ ]:
import duckdb

# Limitar a 1 thread (útil para debugging)
con = duckdb.connect(config={'threads': 1})

## 7. Configurações de Conexão

Configurando opções ao criar conexão.

In [ ]:
import duckdb

with duckdb.connect("file.db") as con:
    con.sql("CREATE TABLE test (i INTEGER)")
    con.sql("INSERT INTO test VALUES (42)")
    con.table("test").show()
# Conexão é fechada automaticamente

## 6. Context Manager (with statement)

Usando context manager para fechar conexão automaticamente.

In [ ]:
import duckdb

# Criar ou conectar a um arquivo de banco de dados
con = duckdb.connect("file.db")
con.sql("CREATE TABLE test (i INTEGER)")
con.sql("INSERT INTO test VALUES (42)")
con.table("test").show()
con.close()

## 5. Conexão com Arquivo de Banco de Dados

Criando ou conectando a um arquivo de banco de dados.

In [ ]:
import duckdb

# Conexão padrão (em memória)
con = duckdb.connect()
con.sql("SELECT 42 AS x").show()

## 4. Conexão em Memória

Criando conexão padrão em memória.

In [ ]:
import duckdb

# Query simples para testar
duckdb.sql("SELECT 42").show()

## 3. Query Simples de Teste

## 2. Imports e Teste Inicial

Importando DuckDB e fazendo um teste simples.

In [ ]:
# Instalação de dependências
%pip install duckdb -q

## 1. Instalação de Dependências

Instalando o DuckDB.